In [1]:
from energyplus.ooep import (
    World,
    #WeatherModel,
    #Report,
    Actuator,
    OutputVariable,
)

from energyplus.dataset.basic import dataset as _epds_

world = World(
    input=World.Specs.Input(
        world=(
            _epds_.models / 'ASHRAE901_OfficeLarge_STD2019_Denver_Chiller205_Detailed.idf'
        ),
        weather=(_epds_.weathers / 'USA_CO_Denver-Aurora-Buckley.AFB.724695_TMY3.epw'),
    ),
    output=World.Specs.Output(
        report=('/tmp/ooep-report-9e1287d2-8e75-4cf5-bbc5-f76580b56a69'),
    ),
    runtime=World.Specs.Runtime(
        #recurring=True,
        #design_day=True,
    ),
)

In [2]:
from energyplus.ooep.addons.rl import (
    SpaceVariableContainer,
    VariableBox,

)

import gymnasium as _gymnasium_
import numpy as _numpy_


env = SpaceVariableContainer(
    action_space=_gymnasium_.spaces.Dict({
        'thermostat': VariableBox(
            low=15., high=20.,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(Actuator.Ref(
            type='Zone Temperature Control',
            control_type='Heating Setpoint',
            key='CORE_MID',
        ))
    }),    
    observation_space=_gymnasium_.spaces.Dict({
        'temperature': VariableBox(
            low=-_numpy_.inf, high=+_numpy_.inf,
            dtype=_numpy_.float32,
            shape=(),
        ).bind(OutputVariable.Ref(
            type='People Air Temperature',
            key='CORE_MID',
        )),
    }),
).__attach__(world)

@world.on('end_zone_timestep_after_zone_reporting')
def _(_):
    env.observation.value
    env.observe()

    # env.action.value = {
    #     'thermostat': 20,
    # }
    # env.act(...)

# world.on('end_zone_timestep_after_zone_reporting', ...)

In [3]:
world.awaitable.run()

<Task pending name='Task-5' coro=<World.run() running at /home/user@AD/lab/EnergyPlus-OOEP/packages/energyplus/ooep/utils/awaitables.py:81>>

In [8]:
for _ in range(10):
    ctx = await world.events['end_zone_timestep_after_zone_reporting'].awaitable(deferred=True)
    print(world['wallclock'].value, env.observe())
    env.act({'thermostat': 20})
    ctx.ack()

1999-04-01 00:14:00+00:00 {'temperature': array(19.979559, dtype=float32)}
1999-04-01 00:29:00+00:00 {'temperature': array(19.999172, dtype=float32)}
1999-04-01 00:44:00+00:00 {'temperature': array(19.709154, dtype=float32)}
1999-04-01 00:59:00+00:00 {'temperature': array(19.468616, dtype=float32)}
1999-04-01 01:14:00+00:00 {'temperature': array(19.95597, dtype=float32)}
1999-04-01 01:29:00+00:00 {'temperature': array(19.998209, dtype=float32)}
1999-04-01 01:44:00+00:00 {'temperature': array(19.69606, dtype=float32)}
1999-04-01 01:59:00+00:00 {'temperature': array(19.45612, dtype=float32)}
1999-04-01 02:14:00+00:00 {'temperature': array(19.95487, dtype=float32)}
1999-04-01 02:29:00+00:00 {'temperature': array(19.998095, dtype=float32)}


In [5]:
#event_ctx, = ctx.input
#event_ctx.event